In [1]:
import pymongo
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
db = client["off"]
collection = db["products"]

data = []
for product in collection.find().limit(50000):
    nutritional_values = product.get('nutriments', {})
    if 'ingredients_text' in product and product['ingredients_text']:
        item = {
            'id': product['_id'],
            'name': product.get('product_name', ''),
            'ingredients': product['ingredients_text'],
            'calories': nutritional_values.get('energy-kcal_100g', 0),
            'proteins': nutritional_values.get('proteins_100g', 0),
            'carbohydrates': nutritional_values.get('carbohydrates_100g', 0),
            'fats': nutritional_values.get('fat_100g', 0)
        }
        data.append(item)

df = pd.DataFrame(data)

df = df.dropna(subset=['name'])
df = df.fillna({'ingredients': '', 'calories': 0, 'proteins': 0, 'carbohydrates': 0, 'fats': 0})

print(f"Nombre de produits après nettoyage : {len(df)}")


Nombre de produits après nettoyage : 48025


In [3]:
for i in range(5):
    print(f"Product {i+1} : {df['name'][i]}")
    print(f"Ingredients : {df['ingredients'][i]}")
    print("")

Product 1 : Chocolat au lait
Ingredients : MODE D'UTILISATION utilisez la ut assaisonner les potes 'les pizzas et lasagnes. Prête o l'emploi. : POURIOOgDE PRODUIT dtmùrts lég Prunes lg 0,3B 4,22 Cascina San Cassiano C.so Piave, 182 - Alba (CN) - Italia www.cascinasancassiano.com

Product 2 :  moutarde au moût de raisin 
Ingredients : eau graines de téguments de moutarde vinaigre de vin rouge sel vin rouge sucre   moût de raisin (6.2%) oignons colorants extraits de carotte et extrait de paprika huile de tournesol son de moutarde sel
(cette _moutarde_ uniquement disponible chez courte paille)

Product 3 : Solène céréales poulet
Ingredients : antioxydant : érythorbate de sodium, colorant : caramel - origine UE), tomate 33,3%, MAYONNAISE 11,1% (huile de colza 78,9%, eau, jaunes d'OEUF 6%, vinaigre, MOUTARDE [eau, graines de MOUTARDE, sel, vinaigre, curcuma], sel, dextrose, stabilisateur : gomme de cellulose, conservateur : sorbate de potassium, colorant : ?-carotène, arôme)

Product 4 : Cr

In [4]:
scaler = StandardScaler()
nutritional_features = ['calories', 'proteins', 'carbohydrates', 'fats']
df[nutritional_features] = scaler.fit_transform(df[nutritional_features])

tfidf = TfidfVectorizer(stop_words='english')
ingredients_tfidf = tfidf.fit_transform(df['ingredients'])

print(f"Shape of the TF-IDF matrix: {ingredients_tfidf.shape}")
print(f"Number of non-zero elements: {ingredients_tfidf.nnz}")
print(f"Sparsity: {100.0 * ingredients_tfidf.nnz / (ingredients_tfidf.shape[0] * ingredients_tfidf.shape[1]):.2f}%")

Shape of the TF-IDF matrix: (48025, 12206)
Number of non-zero elements: 1130753
Sparsity: 0.19%


In [5]:
nutritional_tensor = torch.FloatTensor(df[nutritional_features].values)
ingredients_tensor = torch.FloatTensor(ingredients_tfidf.toarray())

feature_tensor = torch.cat((nutritional_tensor, ingredients_tensor), dim=1)

class ProductEmbedding(torch.nn.Module):
    def __init__(self, input_size, embedding_size):
        super(ProductEmbedding, self).__init__()
        self.embedding = torch.nn.Sequential(
            torch.nn.Linear(input_size, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, embedding_size),
            torch.nn.ReLU()
        )

    def forward(self, x):
        return self.embedding(x)

input_size = feature_tensor.shape[1]
embedding_size = 128
model = ProductEmbedding(input_size, embedding_size)

with torch.no_grad():
    product_embeddings = model(feature_tensor)

print(f"Shape des embeddings : {product_embeddings.shape}")

Shape des embeddings : torch.Size([48025, 128])


In [9]:
nutritional_tensor = torch.FloatTensor(df[nutritional_features].values)
ingredients_tensor = torch.FloatTensor(ingredients_tfidf.toarray())

feature_tensor = torch.cat((nutritional_tensor, ingredients_tensor), dim=1)

print(f"Shape des vecteurs combinés : {feature_tensor.shape}")

def get_recommendations(product_id, top_n=10):
    if product_id not in df['id'].values:
        return "Product not found"

    idx = df.index[df['id'] == product_id].tolist()[0]
    target_embedding = feature_tensor[idx].unsqueeze(0)
    
    similarities = torch.cosine_similarity(target_embedding, feature_tensor)
    
    _, indices = torch.topk(similarities, top_n + 1)
    recommended_indices = indices[1:]  # Exclure le produit lui-même

    return df['name'].iloc[recommended_indices].tolist()


# 104 huile
# 1 chocolat

index_test = 0
test_product_id = df['id'].iloc[index_test]
recommendations = get_recommendations(test_product_id)

print(f"Recommandations pour le produit {test_product_id} ({df['name'].iloc[index_test]}):")
for rec in recommendations:
    print(rec)


Shape des vecteurs combinés : torch.Size([48025, 12210])
Recommandations pour le produit 0000000000000 (Chocolat au lait):
Hickory Harvest, Sesame Sticks
Baked Cheese Crackers
Yogurt Raisin & Peanut Mix
Toffee Pecans
White Cheddar Popcorn
Pop Corn
Mrs. Weinstein's Toffee, Milk Chocolate Squares, Almond
Annie's Organic White Cheddar Popcorn
Chocolate Peanut Clusters
Kowalskis Markets, Chocolate Grahams, Peanut Butter


In [7]:
import random

# Sélectionner 10 produits au hasard
random_product_indices = random.sample(range(len(df)), 10)

# Pour chaque produit aléatoire, afficher les 3 produits les plus similaires
for index in random_product_indices:
    test_product_id = df['id'].iloc[index]
    recommendations = get_recommendations(test_product_id, top_n=3)
    
    print(f"Recommandations pour le produit {test_product_id} ({df['name'].iloc[index]}):")
    for rec in recommendations:
        print(rec)
    print("\n")


Recommandations pour le produit 0016000289987 (Rich & creamy vanilla frosting):
Harina para preparar brownie saboe a chocolate
Bebida Gatorade sabor limonada
Gatorade Sabor Uva


Recommandations pour le produit 0041220849611 (H-e-b, breakfast bread, cinnamon raisin):
H-e-b, breakfast bread, apple strudel
Breakfast Bread
Progresso Italian Style Bread Crumbs


Recommandations pour le produit 0035826055750 (Food lion, jumbo butterfly shrimp):
Popcorn shrimp
Popcorn Shrimp
Popcorn Shrimp


Recommandations pour le produit 0030034094416 (Kettle cooked potato chips):
Giant eagle, kettle cooked original potato chips
Reduced fat original kettle chips, original
Kroger, kettle cooked potato chips, original




KeyboardInterrupt: 